## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key- must put a copy of the config.py file in this folder
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Palmer,US,42.1584,-72.3287,74.01,73,20,5.75,few clouds
1,1,Houma,US,29.5958,-90.7195,84.97,81,22,5.12,few clouds
2,2,Roberval,CA,48.5168,-72.2324,72.23,59,100,12.64,overcast clouds
3,3,Poplar Bluff,US,36.7570,-90.3929,76.78,73,1,4.61,clear sky
4,4,Prince Rupert,CA,54.3161,-130.3201,60.89,82,90,6.91,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 76.58
What is the maximum temperature you would like for your trip? 85.99


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Houma,US,29.5958,-90.7195,84.97,81,22,5.12,few clouds
3,3,Poplar Bluff,US,36.7570,-90.3929,76.78,73,1,4.61,clear sky
6,6,Fortuna,US,40.5982,-124.1573,81.19,60,1,8.01,clear sky
10,10,Opelousas,US,30.5335,-92.0815,80.49,60,1,0.00,clear sky
11,11,Hilo,US,19.7297,-155.0900,76.80,92,99,9.93,light rain
...,...,...,...,...,...,...,...,...,...,...
201,201,Minot,US,48.2325,-101.2963,78.12,49,1,3.00,clear sky
202,202,Gallup,US,35.5281,-108.7426,83.16,26,1,14.97,clear sky
205,205,Oakdale,US,44.9630,-92.9649,80.02,71,1,1.99,clear sky
208,208,Bloomingdale,US,27.8936,-82.2404,85.89,86,90,1.01,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
# they all say 75 or it could be different based on input, but they are all the same which is what matters

City_ID                75
City                   75
Country                75
Lat                    75
Lng                    75
Max Temp               75
Humidity               75
Cloudiness             75
Wind Speed             75
Current Description    75
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Houma,US,84.97,few clouds,29.5958,-90.7195,
3,Poplar Bluff,US,76.78,clear sky,36.7570,-90.3929,
6,Fortuna,US,81.19,clear sky,40.5982,-124.1573,
10,Opelousas,US,80.49,clear sky,30.5335,-92.0815,
11,Hilo,US,76.80,light rain,19.7297,-155.0900,
12,Hamilton,US,76.77,clear sky,39.1834,-84.5333,
18,Kapaa,US,84.52,broken clouds,22.0752,-159.3190,
20,Springville,US,78.67,haze,40.1652,-111.6107,
25,Kahului,US,85.26,few clouds,20.8947,-156.4700,
31,Glendive,US,80.22,broken clouds,47.1053,-104.7125,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    try:        

        # 6e. Make request and retrieve the JSON data from the search.
        hotels = requests.get(base_url, params=params).json()

        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        city_name = row["City"]        
        print("Hotel not found for city = " + city_name + " ... skipping.")
        

Hotel not found for city = Elko ... skipping.
Hotel not found for city = Winona ... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
# hotel_df.count()

clean_hotel_df = hotel_df.dropna()

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Houma,US,84.97,few clouds,29.5958,-90.7195,Courtyard by Marriott Houma
3,Poplar Bluff,US,76.78,clear sky,36.7570,-90.3929,Holiday Inn Poplar Bluff
6,Fortuna,US,81.19,clear sky,40.5982,-124.1573,Super 8 by Wyndham Fortuna
10,Opelousas,US,80.49,clear sky,30.5335,-92.0815,"Holiday Inn & Suites Opelousas, an IHG Hotel"
11,Hilo,US,76.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
201,Minot,US,78.12,clear sky,48.2325,-101.2963,Noble Inn
202,Gallup,US,83.16,clear sky,35.5281,-108.7426,El Capitan Motel
205,Oakdale,US,80.02,clear sky,44.9630,-92.9649,Best Western Regency Plaza Hotel - St. Paul East
208,Bloomingdale,US,85.89,overcast clouds,27.8936,-82.2404,Travel Club for Free


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
hotel_info


['\n<dl>\n<dt>Hotel Name</dt><dd>Courtyard by Marriott Houma</dd>\n<dt>City</dt><dd>Houma</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Weather Description</dt><dd>few clouds and 84.97 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Holiday Inn Poplar Bluff</dd>\n<dt>City</dt><dd>Poplar Bluff</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Weather Description</dt><dd>clear sky and 76.78 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Super 8 by Wyndham Fortuna</dd>\n<dt>City</dt><dd>Fortuna</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Weather Description</dt><dd>clear sky and 81.19 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Holiday Inn & Suites Opelousas, an IHG Hotel</dd>\n<dt>City</dt><dd>Opelousas</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Weather Description</dt><dd>clear sky and 80.49 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hilo Hawaiian Hotel</dd>\n<dt>City</dt><dd>Hilo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Weather Description</dt><dd>light rain and 76.8 °F</dd>\n</dl>\n',
 '

In [27]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(40.0, -75.0), zoom_level=6)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [28]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# must also save the map itself to the folder and call it WeatherPy_vacation_map.png
# Saving the map using the download button above did not show the markers text so
# so I am also saving another screen capture in the same folder 
# the file is "./World_Weather_Analysis/Vacation_Search/gwu20210807_challenge_deliverable2_map_image_with_markers.JPG"